## ACRONYM Visual test

In [1]:
import sys,os
import json
import trimesh
import argparse
import numpy as np

from acronym_tools import load_mesh, load_grasps, create_gripper_marker

sys.path

['/home/juju/contact_graspnet/contact_graspnet_study/acronym_test',
 '/home/juju/contact_graspnet/contact_graspnet_study/acronym_test',
 '/home/juju/contact_graspnet',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python38.zip',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8/lib-dynload',
 '',
 '/home/juju/.local/lib/python3.8/site-packages',
 '/home/juju/.local/lib/python3.8/site-packages/pycocotools-2.0-py3.8-linux-x86_64.egg',
 '/home/juju/anaconda3/envs/mmpose/lib/python3.8/site-packages',
 '/home/juju/anaconda3/envs/contact_graspnet/lib/python3.8/site-packages',
 '/home/juju/acronym']

### Parsing 


In [2]:
def make_parser():
    parser = argparse.ArgumentParser(
        description="Visualize grasps from the dataset.",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    parser.add_argument("input", nargs="+", help="HDF5 or JSON Grasp file(s).")
    parser.add_argument(
        "--num_grasps", type=int, default=20, help="Number of grasps to show."
    )
    parser.add_argument(
        "--mesh_root", default=".", help="Directory used for loading meshes."
    )
    
    return parser

import easydict

def custom_parser():
    args = easydict.EasyDict({
        "input" : ["/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5", "/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5"],
        "num_grasps" : 5,
        "mesh_root" : '/home/juju/acronym/data/examples/'
    })
    return args

In [3]:
args = custom_parser()
args

{'input': ['/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5',
  '/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5'],
 'num_grasps': 5,
 'mesh_root': '/home/juju/acronym/data/examples/'}

### load mesh

In [4]:
import os
import json
import h5py
import trimesh
import trimesh.path
import trimesh.transformations as tra
import numpy as np

filename = args.input[0] 
filename.endswith(".h5")

scale=None

data = h5py.File(filename, "r")
mesh_fname = data["object/file"][()].decode('utf-8')
mesh_scale = data["object/scale"][()] 

obj_mesh = trimesh.load(os.path.join(args.mesh_root, mesh_fname))
obj_mesh = obj_mesh.apply_scale(mesh_scale)

In [5]:
mesh_fname

'meshes/Mug/10f6e09036350e92b3f21f1137c3c347.obj'

### load grasps

In [6]:
filename = args.input[0] 
filename.endswith(".h5")

data = h5py.File(filename, "r")
T = np.array(data["grasps/transforms"])
success = np.array(data["grasps/qualities/flex/object_in_gripper"])

In [7]:
success

array([0, 0, 0, ..., 1, 1, 0])

### create_gripper_marker

In [8]:
# successful_grasps = [
#     create_gripper_marker(color=[0, 255, 0]).apply_transform(t)
#     for t in T[np.random.choice(np.where(success == 1)[0], args.num_grasps)]
# ]
# failed_grasps = [
#     create_gripper_marker(color=[255, 0, 0]).apply_transform(t)
#     for t in T[np.random.choice(np.where(success == 0)[0], args.num_grasps)]
# ]

# color=[0, 0, 255], tube_radius=0.001, sections=6

cfl = trimesh.creation.cylinder(
    radius=0.002,
    sections=sections,
    segment=[
        [4.10000000e-02, -7.27595772e-12, 6.59999996e-02],
        [4.10000000e-02, -7.27595772e-12, 1.12169998e-01],
    ],
)
cfr = trimesh.creation.cylinder(
    radius=0.002,
    sections=sections,
    segment=[
        [-4.100000e-02, -7.27595772e-12, 6.59999996e-02],
        [-4.100000e-02, -7.27595772e-12, 1.12169998e-01],
    ],
)
cb1 = trimesh.creation.cylinder(
    radius=0.002, sections=sections, segment=[[0, 0, 0], [0, 0, 6.59999996e-02]]
)
cb2 = trimesh.creation.cylinder(
    radius=0.002,
    sections=sections,
    segment=[[-4.100000e-02, 0, 6.59999996e-02], [4.100000e-02, 0, 6.59999996e-02]],
)

tmp = trimesh.util.concatenate([cb1, cb2, cfr, cfl])
tmp.visual.face_colors = color

NameError: name 'sections' is not defined

In [9]:
mesh_fname

'meshes/Mug/10f6e09036350e92b3f21f1137c3c347.obj'

In [10]:
for f in args.input:
    # load object mesh
    print(f)
    obj_mesh = load_mesh(f, mesh_root_dir=args.mesh_root)

    # get transformations and quality of all simulated grasps
    T, success = load_grasps(f)

    # create visual markers for grasps
    successful_grasps = [
        create_gripper_marker(color=[0, 255, 0]).apply_transform(t)
        for t in T[np.random.choice(np.where(success == 1)[0], args.num_grasps)]
    ]
    failed_grasps = [
        create_gripper_marker(color=[255, 0, 0]).apply_transform(t)
        for t in T[np.random.choice(np.where(success == 0)[0], args.num_grasps)]
    ]

    Scene = trimesh.Scene([obj_mesh] + successful_grasps + failed_grasps)


/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5
/home/juju/acronym/data/examples/grasps/Mug_10f6e09036350e92b3f21f1137c3c347_0.0002682457830986903.h5


In [11]:
# trimesh.viewer.notebook.in_notebook()
trimesh.viewer.notebook.scene_to_notebook(Scene)

AttributeError: module 'trimesh' has no attribute 'viewer'

In [113]:
obj_mesh.bounds

array([[-0.03810737, -0.06301068,  0.05142949],
       [ 0.03806329,  0.03802388,  0.15249297]])